In [8]:
using PPLM

In [9]:
tokenizer, model = PPLM.get_gpt2();

┌ Warning: load from base: prediction layer not found in state: initialized.
└ @ Transformers.HuggingFace /home/adarshkumar712/.julia/packages/Transformers/3YgSd/src/huggingface/models/models.jl:102
┌ Warning: typename(Transformers.HuggingFace.HGFGPT2Attention) doesn't have field bias.
└ @ Transformers.HuggingFace /home/adarshkumar712/.julia/packages/Transformers/3YgSd/src/huggingface/models/models.jl:47
┌ Warning: typename(Transformers.HuggingFace.HGFGPT2Attention) doesn't have field bias.
└ @ Transformers.HuggingFace /home/adarshkumar712/.julia/packages/Transformers/3YgSd/src/huggingface/models/models.jl:47
┌ Warning: typename(Transformers.HuggingFace.HGFGPT2Attention) doesn't have field bias.
└ @ Transformers.HuggingFace /home/adarshkumar712/.julia/packages/Transformers/3YgSd/src/huggingface/models/models.jl:47
┌ Warning: typename(Transformers.HuggingFace.HGFGPT2Attention) doesn't have field bias.
└ @ Transformers.HuggingFace /home/adarshkumar712/.julia/packages/Transformers/3YgSd/s

In [10]:
text, label = PPLM.load_data_from_csv("./../Datasets/train.csv";text_col="comment_text", label_col="toxic", delim=","); # replace with appropriate Dataset path, text_col and label_col

In [11]:
discrim = PPLM.get_discriminator(model; class_size=2);

In [12]:
ind_true = findall(label.==true);
ind_false = findall(label.==false);

In [13]:
using StatsBase
using Random

In [14]:
rng = MersenneTwister(1234);
true_sample = shuffle(rng, ind_true)[1:15000];
false_sample = shuffle(rng, ind_false)[1:15000];

In [15]:
all_samples = cat(true_sample, false_sample, dims=1);

In [16]:
text_reduced = text[all_samples];
label_reduced = label[all_samples];

In [17]:
using CUDA
CUDA.allowscalar(false)

In [18]:
args = PPLM.HyperParams(lr=5e-6,classification_type="MultiClass", epochs=50)

PPLM.HyperParams
  batchsize: Int64 8
  eos_token: Int64 50527
  pad_token: Int64 0
  embed_size: Int64 768
  classification_type: String "MultiClass"
  target_class: Int64 1
  num_classes: Int64 2
  cached: Bool true
  lr: Float64 5.0e-6
  epochs: Int64 50
  log_interval: Int64 100


# Method 1

In [19]:
PPLM.train_discriminator(text_reduced, label_reduced, 8, "Multiclass", 2; lr=5e-6, discrim=discrim, tokenizer=tokenizer, args=args, train_size=0.85, epochs=50);

Creating Cache of data and Data Loader...

Computing...100%|███████████████████████████████████████| Time: 0:30:03m30m
Computing...100%|███████████████████████████████████████| Time: 0:01:41


Training starting.........

Epoch #1 : Step #100 : NLL Loss: 7.163510203361511
Epoch #1 : Step #200 : NLL Loss: 6.954627327919006
Epoch #1 : Step #300 : NLL Loss: 6.925139118830363
Epoch #1 : Step #400 : NLL Loss: 6.951619546711445
Epoch #1 : Step #500 : NLL Loss: 6.8917058465480805
Epoch #1 : Step #600 : NLL Loss: 6.749353889028232
Epoch #1 : Step #700 : NLL Loss: 6.6492555820941925
Epoch #1 : Step #800 : NLL Loss: 6.543093764483928
Epoch #1 : Step #900 : NLL Loss: 6.435024647315343
Epoch #1 : Step #1000 : NLL Loss: 6.337236048698426
Epoch #1 : Step #1100 : NLL Loss: 6.212964235219088
Epoch #1 : Step #1200 : NLL Loss: 6.116989037195841
Epoch #1 : Step #1300 : NLL Loss: 5.9921144500145544
Epoch #1 : Step #1400 : NLL Loss: 5.870880446859768
Epoch #1 : Step #1500 : NLL Loss: 5.777817563255628
Epoch #1 : Step #1600 : NLL Loss: 5.665871746875346
Epoch #1 : Step #1700 : NLL Loss: 5.560679710787885
Epoch #1 : Step #1800 : NLL Loss: 5.4437543655435245
Epoch #1 : Step #1900 : NLL Loss: 5.33173

Epoch #5 : Step #3100 : NLL Loss: 0.5576759557954727
Loss Epoch #5 : 3188 NLL Loss: 0.556316845801424

Epoch #6 : Step #100 : NLL Loss: 0.5357412496209144
Epoch #6 : Step #200 : NLL Loss: 0.5495168827474117
Epoch #6 : Step #300 : NLL Loss: 0.5425607279936473
Epoch #6 : Step #400 : NLL Loss: 0.5425075631588697
Epoch #6 : Step #500 : NLL Loss: 0.5390195402503014
Epoch #6 : Step #600 : NLL Loss: 0.532209989875555
Epoch #6 : Step #700 : NLL Loss: 0.5357000788194792
Epoch #6 : Step #800 : NLL Loss: 0.5293076775968075
Epoch #6 : Step #900 : NLL Loss: 0.5243838971190983
Epoch #6 : Step #1000 : NLL Loss: 0.5209631095230579
Epoch #6 : Step #1100 : NLL Loss: 0.5208315500887958
Epoch #6 : Step #1200 : NLL Loss: 0.5198536813507477
Epoch #6 : Step #1300 : NLL Loss: 0.5197337271158512
Epoch #6 : Step #1400 : NLL Loss: 0.5169973816190447
Epoch #6 : Step #1500 : NLL Loss: 0.5153426946997642
Epoch #6 : Step #1600 : NLL Loss: 0.513513153064996
Epoch #6 : Step #1700 : NLL Loss: 0.5125086650953573
Epoch #

Epoch #10 : Step #2600 : NLL Loss: 0.37314182400703433
Epoch #10 : Step #2700 : NLL Loss: 0.37259868385615175
Epoch #10 : Step #2800 : NLL Loss: 0.37281958829079354
Epoch #10 : Step #2900 : NLL Loss: 0.3727307544802797
Epoch #10 : Step #3000 : NLL Loss: 0.37223725534478824
Epoch #10 : Step #3100 : NLL Loss: 0.37242506270447084
Loss Epoch #10 : 3188 NLL Loss: 0.3727460496940906

Epoch #11 : Step #100 : NLL Loss: 0.3812038683891296
Epoch #11 : Step #200 : NLL Loss: 0.3669163554906845
Epoch #11 : Step #300 : NLL Loss: 0.36636526574691136
Epoch #11 : Step #400 : NLL Loss: 0.3627946697920561
Epoch #11 : Step #500 : NLL Loss: 0.3594117218852043
Epoch #11 : Step #600 : NLL Loss: 0.35777595673998197
Epoch #11 : Step #700 : NLL Loss: 0.36045928678342276
Epoch #11 : Step #800 : NLL Loss: 0.36709382981061933
Epoch #11 : Step #900 : NLL Loss: 0.3684851606355773
Epoch #11 : Step #1000 : NLL Loss: 0.36693612244725227
Epoch #11 : Step #1100 : NLL Loss: 0.3672542225230824
Epoch #11 : Step #1200 : NLL 

...

Epoch #29 : Step #2700 : NLL Loss: 0.28504962884717516
Epoch #29 : Step #2800 : NLL Loss: 0.28456944309175014
Epoch #29 : Step #2900 : NLL Loss: 0.2850402404213774
Epoch #29 : Step #3000 : NLL Loss: 0.28543336428205174
Epoch #29 : Step #3100 : NLL Loss: 0.28456861234480335
Loss Epoch #29 : 3188 NLL Loss: 0.285109181756357

Epoch #30 : Step #100 : NLL Loss: 0.2791083097457886
Epoch #30 : Step #200 : NLL Loss: 0.28846052184700965
Epoch #30 : Step #300 : NLL Loss: 0.28265669653813047
Epoch #30 : Step #400 : NLL Loss: 0.2842049807310104
Epoch #30 : Step #500 : NLL Loss: 0.27940057188272477
Epoch #30 : Step #600 : NLL Loss: 0.27713426560163495
Epoch #30 : Step #700 : NLL Loss: 0.28014902723687035
Epoch #30 : Step #800 : NLL Loss: 0.27863964293152094
Epoch #30 : Step #900 : NLL Loss: 0.28056484921111
Epoch #30 : Step #1000 : NLL Loss: 0.27955946877598764
Epoch #30 : Step #1100 : NLL Loss: 0.27898378960110926
Epoch #30 : Step #1200 : NLL Loss: 0.2767645520468553
Epoch #30 : Step #1300 : NLL L

Epoch #34 : Step #1900 : NLL Loss: 0.2757549960440711
Epoch #34 : Step #2000 : NLL Loss: 0.27666096641868354
Epoch #34 : Step #2100 : NLL Loss: 0.27683255931451206
Epoch #34 : Step #2200 : NLL Loss: 0.2783753036030314
Epoch #34 : Step #2300 : NLL Loss: 0.27797556066642637
Epoch #34 : Step #2400 : NLL Loss: 0.2777432103889684
Epoch #34 : Step #2500 : NLL Loss: 0.2774116170704365
Epoch #34 : Step #2600 : NLL Loss: 0.2785684474099141
Epoch #34 : Step #2700 : NLL Loss: 0.27886282806595164
Epoch #34 : Step #2800 : NLL Loss: 0.27872266894472497
Epoch #34 : Step #2900 : NLL Loss: 0.278282491569889
Epoch #34 : Step #3000 : NLL Loss: 0.27860481462379294
Epoch #34 : Step #3100 : NLL Loss: 0.2789381901246886
Loss Epoch #34 : 3188 NLL Loss: 0.27904916141027936

Epoch #35 : Step #100 : NLL Loss: 0.27583352714776993
Epoch #35 : Step #200 : NLL Loss: 0.2749754309654236
Epoch #35 : Step #300 : NLL Loss: 0.2793347628911336
Epoch #35 : Step #400 : NLL Loss: 0.27652837850153444
Epoch #35 : Step #500 : NL

Epoch #39 : Step #1200 : NLL Loss: 0.2760102513929208
Epoch #39 : Step #1300 : NLL Loss: 0.2764447035010044
Epoch #39 : Step #1400 : NLL Loss: 0.275773047719683
Epoch #39 : Step #1500 : NLL Loss: 0.274072335600853
Epoch #39 : Step #1600 : NLL Loss: 0.27500954311341047
Epoch #39 : Step #1700 : NLL Loss: 0.2744770018493428
Epoch #39 : Step #1800 : NLL Loss: 0.2746285655928983
Epoch #39 : Step #1900 : NLL Loss: 0.27400897742886293
Epoch #39 : Step #2000 : NLL Loss: 0.2734806661605835
Epoch #39 : Step #2100 : NLL Loss: 0.2718750817860876
Epoch #39 : Step #2200 : NLL Loss: 0.27161331950263545
Epoch #39 : Step #2300 : NLL Loss: 0.27240407183118487
Epoch #39 : Step #2400 : NLL Loss: 0.2730951209863027
Epoch #39 : Step #2500 : NLL Loss: 0.2737633843660355
Epoch #39 : Step #2600 : NLL Loss: 0.2743750479358893
Epoch #39 : Step #2700 : NLL Loss: 0.27489003870222306
Epoch #39 : Step #2800 : NLL Loss: 0.27353152263377395
Epoch #39 : Step #2900 : NLL Loss: 0.2729113427318376
Epoch #39 : Step #3000 :

Epoch #44 : Step #500 : NLL Loss: 0.28487668734788896
Epoch #44 : Step #600 : NLL Loss: 0.2804603813091914
Epoch #44 : Step #700 : NLL Loss: 0.2758538685526167
Epoch #44 : Step #800 : NLL Loss: 0.2752643288299441
Epoch #44 : Step #900 : NLL Loss: 0.2777780834502644
Epoch #44 : Step #1000 : NLL Loss: 0.27688266360759733
Epoch #44 : Step #1100 : NLL Loss: 0.2771696314757521
Epoch #44 : Step #1200 : NLL Loss: 0.2755589907368024
Epoch #44 : Step #1300 : NLL Loss: 0.27399270376333823
Epoch #44 : Step #1400 : NLL Loss: 0.2746824594267777
Epoch #44 : Step #1500 : NLL Loss: 0.2737137125531832
Epoch #44 : Step #1600 : NLL Loss: 0.27443694349378345
Epoch #44 : Step #1700 : NLL Loss: 0.27379699994536005
Epoch #44 : Step #1800 : NLL Loss: 0.27273143225246005
Epoch #44 : Step #1900 : NLL Loss: 0.26964213104624496
Epoch #44 : Step #2000 : NLL Loss: 0.2697739043235779
Epoch #44 : Step #2100 : NLL Loss: 0.26989519838775905
Epoch #44 : Step #2200 : NLL Loss: 0.2708981900188056
Epoch #44 : Step #2300 : 

Epoch #48 : Step #3000 : NLL Loss: 0.26875518080592153
Epoch #48 : Step #3100 : NLL Loss: 0.2674621000309144
Loss Epoch #48 : 3188 NLL Loss: 0.2683400279156387

Epoch #49 : Step #100 : NLL Loss: 0.25882813841104507
Epoch #49 : Step #200 : NLL Loss: 0.2674956977367401
Epoch #49 : Step #300 : NLL Loss: 0.26706644902626675
Epoch #49 : Step #400 : NLL Loss: 0.2619867607206106
Epoch #49 : Step #500 : NLL Loss: 0.2668941687941551
Epoch #49 : Step #600 : NLL Loss: 0.26126374741395314
Epoch #49 : Step #700 : NLL Loss: 0.2629385876229831
Epoch #49 : Step #800 : NLL Loss: 0.26294676076620815
Epoch #49 : Step #900 : NLL Loss: 0.2610928330818812
Epoch #49 : Step #1000 : NLL Loss: 0.2614251511991024
Epoch #49 : Step #1100 : NLL Loss: 0.2633780268647454
Epoch #49 : Step #1200 : NLL Loss: 0.2626944573720296
Epoch #49 : Step #1300 : NLL Loss: 0.26206786112143443
Epoch #49 : Step #1400 : NLL Loss: 0.2631879593006202
Epoch #49 : Step #1500 : NLL Loss: 0.26512648129463195
Epoch #49 : Step #1600 : NLL Los

# Method 2

In [30]:
(train_x, train_y), (test_x, test_y) = PPLM.splitobs((text_reduced, label_reduced); at=0.8)
        
train_loader = PPLM.load_cached_data(discrim, train_x, train_y, tokenizer; truncate=true, classification_type="Multiclass");

test_loader = PPLM.load_cached_data(discrim, test_x, test_y, tokenizer; truncate=true, classification_type="Multiclass");


In [31]:
PPLM.train!(discrim, train_loader; args=args)

Training starting.........

Epoch #1 : Step #100 : NLL Loss: 1.9753262543678283
Epoch #1 : Step #200 : NLL Loss: 1.9231716027855874
Epoch #1 : Step #300 : NLL Loss: 1.8749549571673076
Epoch #1 : Step #400 : NLL Loss: 1.8426343695819378
Epoch #1 : Step #500 : NLL Loss: 1.8339375865459442
Epoch #1 : Step #600 : NLL Loss: 1.7932911743720372
Epoch #1 : Step #700 : NLL Loss: 1.7661278000899723
Epoch #1 : Step #800 : NLL Loss: 1.7534843473881483
Epoch #1 : Step #900 : NLL Loss: 1.7093081312709384
Epoch #1 : Step #1000 : NLL Loss: 1.6702592520415782
Epoch #1 : Step #1100 : NLL Loss: 1.6479249478199265
Epoch #1 : Step #1200 : NLL Loss: 1.6145438824842373
Epoch #1 : Step #1300 : NLL Loss: 1.5371075125267872
Epoch #1 : Step #1400 : NLL Loss: 1.4313458395696113
Epoch #1 : Step #1500 : NLL Loss: 1.3402320544868707
Epoch #1 : Step #1600 : NLL Loss: 1.2606613345770166
Epoch #1 : Step #1700 : NLL Loss: 1.1902108608580688
Epoch #1 : Step #1800 : NLL Loss: 1.1275494936646686
Epoch #1 : Step #1900 : NLL

Epoch #7 : Step #200 : NLL Loss: 0.44689521230757234
Epoch #7 : Step #300 : NLL Loss: 0.42431444600224494
Epoch #7 : Step #400 : NLL Loss: 0.4097029895707965
Epoch #7 : Step #500 : NLL Loss: 0.4040096550285816
Epoch #7 : Step #600 : NLL Loss: 0.3890973145514727
Epoch #7 : Step #700 : NLL Loss: 0.3797350714675018
Epoch #7 : Step #800 : NLL Loss: 0.37626804592087865
Epoch #7 : Step #900 : NLL Loss: 0.36204471967286533
Epoch #7 : Step #1000 : NLL Loss: 0.3494673364497721
Epoch #7 : Step #1100 : NLL Loss: 0.3422479705072262
Epoch #7 : Step #1200 : NLL Loss: 0.3323555577515314
Epoch #7 : Step #1300 : NLL Loss: 0.33474427888313163
Epoch #7 : Step #1400 : NLL Loss: 0.3420818977723164
Epoch #7 : Step #1500 : NLL Loss: 0.347988312723736
Epoch #7 : Step #1600 : NLL Loss: 0.3500071258866228
Epoch #7 : Step #1700 : NLL Loss: 0.35234835523235447
Epoch #7 : Step #1800 : NLL Loss: 0.3514699290630718
Epoch #7 : Step #1900 : NLL Loss: 0.3484545923318518
Epoch #7 : Step #2000 : NLL Loss: 0.3465593778733

In [ ]:
PPLM.test!(discrim, test_loader; args=args)

# Save Discriminator

In [20]:
PPLM.save_discriminator(discrim, "toxicity"; file_name="toxicity_classifier_head_768.bson")

Saving classifier head weights for the discriminator to /home/adarshkumar712/Projects/PPLM_1/src/./pretrained_discriminators/toxicity_classifier_head_768.bson


┌ Warning: Saving classifier without Hyperparameter information
└ @ PPLM /home/adarshkumar712/Projects/PPLM_1/src/discriminator.jl:133


Discriminator saved successfully

# Load the saved Discriminator

In [31]:
discrim = PPLM.get_discriminator(model; load_from_pretrained=true, discrim="toxicity", class_size=2);